In [247]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer


import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer

In [248]:
np.random.seed(42)

In [249]:
raw_dataset = pd.read_csv("data/file.csv", index_col=0).to_numpy()

In [250]:
np.random.shuffle(raw_dataset)

In [251]:
WHOLE_DATASET_LEN = 20000
lb = LabelBinarizer()
X, y  = raw_dataset[:, 0], lb.fit_transform(raw_dataset[:, 1])
X, y  = X[:WHOLE_DATASET_LEN], y[:WHOLE_DATASET_LEN]

In [252]:
X_train, X_test = X[:int(WHOLE_DATASET_LEN*0.7)], X[int(WHOLE_DATASET_LEN*0.7):]
y_train, y_test = y[:int(WHOLE_DATASET_LEN*0.7)], y[int(WHOLE_DATASET_LEN*0.7):]

In [253]:
test_set_len = int(len(X_test) * 0.7)
X_test, X_valid = X_test[:test_set_len], X_test[test_set_len:]
y_test, y_valid = y_test[:test_set_len], y_test[test_set_len:]

# BERT

In [254]:
#tiny bert l=2 h=128
tiny_bert_model_url = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2"
bert_preprocessor_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"

In [255]:
def build_bert_model(preprocessor_url, model_url):
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(bert_preprocessor_url, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tiny_bert_model_url, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(3, activation="softmax", name='classifier')(net)
    return tf.keras.Model(text_input, net)

In [256]:
tiny_bert_model = build_bert_model(bert_preprocessor_url, tiny_bert_model_url)

In [257]:
loss = tf.keras.losses.CategoricalCrossentropy()
metric = tf.keras.metrics.CategoricalCrossentropy()

In [258]:
epochs = 8 
steps_pre_epoch = len(X_train) 
num_train_steps = steps_pre_epoch * epochs
num_warmup_steps = int(0.1 * num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(
    init_lr = init_lr,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    optimizer_type='adamw'
)

In [259]:
tiny_bert_model.compile(optimizer=optimizer, loss=loss, metrics=[metric, 'accuracy'])

In [260]:
tiny_bert_model_history = tiny_bert_model.fit(x=X_train, y=y_train,  validation_data=(X_valid, y_valid), epochs=epochs)

Epoch 1/8
438/438 [==============================] - 71s 159ms/step - loss: 1.2780 - categorical_crossentropy: 1.2780 - accuracy: 0.3296 - val_loss: 1.1253 - val_categorical_crossentropy: 1.1253 - val_accuracy: 0.4217
Epoch 2/8
438/438 [==============================] - 68s 155ms/step - loss: 1.1074 - categorical_crossentropy: 1.1074 - accuracy: 0.4366 - val_loss: 1.0473 - val_categorical_crossentropy: 1.0473 - val_accuracy: 0.4811
Epoch 3/8
438/438 [==============================] - 70s 161ms/step - loss: 1.0585 - categorical_crossentropy: 1.0585 - accuracy: 0.4812 - val_loss: 1.0110 - val_categorical_crossentropy: 1.0110 - val_accuracy: 0.5078
Epoch 4/8
438/438 [==============================] - 66s 150ms/step - loss: 1.0280 - categorical_crossentropy: 1.0280 - accuracy: 0.5012 - val_loss: 0.9896 - val_categorical_crossentropy: 0.9896 - val_accuracy: 0.5344
Epoch 5/8
438/438 [==============================] - 65s 149ms/step - loss: 1.0140 - categorical_crossentropy: 1.0140 - accuracy

In [261]:
tiny_bert_model.evaluate(X_test,y_test)

132/132 [==============================] - 5s 41ms/step - loss: 0.8046 - categorical_crossentropy: 0.8046 - accuracy: 0.6350


[0.8046244382858276, 0.8046244382858276, 0.6349999904632568]

In [262]:
mini_bert_model_url = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/2"

In [263]:
mini_bert_model = build_bert_model(bert_preprocessor_url, tiny_bert_model_url)


In [264]:
mini_bert_model.compile(optimizer=optimizer, loss=loss, metrics=[metric, 'accuracy'])

In [265]:
mini_bert_model_history = mini_bert_model.fit(x=X_train, y=y_train,  validation_data=(X_valid, y_valid), epochs=epochs)

Epoch 1/8
438/438 [==============================] - 70s 155ms/step - loss: 1.0107 - categorical_crossentropy: 0.9631 - accuracy: 0.5151 - val_loss: 0.8908 - val_categorical_crossentropy: 0.8908 - val_accuracy: 0.5850
Epoch 2/8
438/438 [==============================] - 68s 156ms/step - loss: 0.8729 - categorical_crossentropy: 0.8729 - accuracy: 0.5975 - val_loss: 0.7948 - val_categorical_crossentropy: 0.7948 - val_accuracy: 0.6594
Epoch 3/8
438/438 [==============================] - 68s 156ms/step - loss: 0.7885 - categorical_crossentropy: 0.7885 - accuracy: 0.6434 - val_loss: 0.7072 - val_categorical_crossentropy: 0.7072 - val_accuracy: 0.6789
Epoch 4/8
438/438 [==============================] - 66s 151ms/step - loss: 0.7240 - categorical_crossentropy: 0.7240 - accuracy: 0.6820 - val_loss: 0.6733 - val_categorical_crossentropy: 0.6733 - val_accuracy: 0.7028
Epoch 5/8
438/438 [==============================] - 65s 147ms/step - loss: 0.6782 - categorical_crossentropy: 0.6782 - accuracy

In [266]:
mini_bert_model.evaluate(X_test,y_test)

132/132 [==============================] - 5s 41ms/step - loss: 0.5647 - categorical_crossentropy: 0.5647 - accuracy: 0.7729


[0.564741849899292, 0.564741849899292, 0.772857129573822]

In [267]:
medium_bert_model_url = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/2"

In [268]:

medium_bert_model = build_bert_model(bert_preprocessor_url, tiny_bert_model_url)

In [269]:

medium_bert_model.compile(optimizer=optimizer, loss=loss, metrics=[metric, 'accuracy'])


In [270]:
medium_bert_model_history = mini_bert_model.fit(x=X_train, y=y_train,  validation_data=(X_valid, y_valid), epochs=epochs)

Epoch 1/8
438/438 [==============================] - 66s 151ms/step - loss: 0.5295 - categorical_crossentropy: 0.5295 - accuracy: 0.7829 - val_loss: 0.5404 - val_categorical_crossentropy: 0.5404 - val_accuracy: 0.7828
Epoch 2/8
438/438 [==============================] - 67s 153ms/step - loss: 0.4948 - categorical_crossentropy: 0.4948 - accuracy: 0.8014 - val_loss: 0.5317 - val_categorical_crossentropy: 0.5317 - val_accuracy: 0.7967
Epoch 3/8
438/438 [==============================] - 66s 151ms/step - loss: 0.4695 - categorical_crossentropy: 0.4695 - accuracy: 0.8144 - val_loss: 0.5192 - val_categorical_crossentropy: 0.5192 - val_accuracy: 0.7994
Epoch 4/8
438/438 [==============================] - 67s 152ms/step - loss: 0.4445 - categorical_crossentropy: 0.4445 - accuracy: 0.8224 - val_loss: 0.5132 - val_categorical_crossentropy: 0.5132 - val_accuracy: 0.8111
Epoch 5/8
438/438 [==============================] - 67s 153ms/step - loss: 0.4216 - categorical_crossentropy: 0.4216 - accuracy

In [272]:
medium_bert_model.evaluate(X_test,y_test)

132/132 [==============================] - 6s 42ms/step - loss: 1.1147 - categorical_crossentropy: 1.1147 - accuracy: 0.3960


[1.1146581172943115, 1.1146581172943115, 0.39595237374305725]

# Biblio
https://www.tensorflow.org/text/tutorials/classify_text_with_bert